In [1]:
import json
from flask import Flask
from flask_cors import CORS, cross_origin
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
# MAGRES database
client = MongoClient(host='wigner.esc.rl.ac.uk')
ccpnc = client.ccpnc

In [3]:
def findById(fileID):
    return ccpnc.magresFiles.find({'_id': ObjectId(fileID)})

In [15]:
ff = ccpnc.magresData.find({"chemname": "Ethanol"})
# Get the first magres file in the query
mfiles = []
for f in ff:
    print f['chemname']
    mfile = findById(f['magresFilesID'])
    for m in mfile:
        mfiles.append(m['magres'])

Ethanol


In [33]:
ff = ccpnc.magresData.find({"orcid.path": "0000-0003-4851-1144"})
for f in ff:
    print f['orcid']

{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
{u'path': u'0000-0003-4851-1144', u'host': u'orcid.org', u'uri': u'http://orcid.org/0000-0003-4851-1144'}
